In [13]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from time import perf_counter
from itertools import combinations, product
from sortedcontainers import SortedSet
import itertools
import copy

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [14]:
def random_search(distance_matrix):
    n = len(distance_matrix)
    
    solution = list(range(n))
    np.random.shuffle(solution)
    
    return solution[:(n//2)]



def get_distance_matrix(df):
    coords = df[['x', 'y']].to_numpy()
    
    distance_matrix = np.round(squareform(pdist(coords, 'euclidean')))
    np.fill_diagonal(distance_matrix, 0)
    
    return distance_matrix, df['costs'].to_numpy()



def get_total_cost(solution, distance_matrix, costs):
    total_cost = 0
    
    for i in range(len(solution)-1):
        total_cost += distance_matrix[solution[i], solution[i+1]] + costs[solution[i+1]]
        
    total_cost += distance_matrix[solution[-1], solution[0]] + costs[solution[0]]
        
    return total_cost

In [15]:
def compute_inter_move_delta(solution, distance_matrix, costs, idx, new_node):
    n = len(solution)
    new_solution = solution.copy()
    old_node = solution[idx]

    new = (costs[new_node] +
            distance_matrix[new_solution[idx-1], new_node] +
            distance_matrix[new_node, new_solution[(idx+1)%n]])

    old = (costs[old_node] +
             distance_matrix[new_solution[idx-1], old_node] +
             distance_matrix[old_node, new_solution[(idx+1)%n]])

    delta = new - old
    new_solution[idx] = new_node

    return new_solution, delta

In [16]:
######## DON'T REMOVE

# def compute_intra_move_delta(solution, distance_matrix, indices, backward=False):
#     n = len(solution)
#     new_solution = solution.copy()
#     i, j = indices
    
#     if i >= j: raise Exception('Wrong indeces, i >= j')
#     if j >= n: raise Exception('Wrong indeces, j >= n')
    
#     if backward:
#         if (i == 0 and j in (n-1, n-2)) or (j == n-1 and i in (0, 1)):
#             return new_solution, 0
#         new = distance_matrix[new_solution[i], new_solution[(j+1)%n]] + distance_matrix[new_solution[j], new_solution[i-1]]
#         old = distance_matrix[new_solution[i-1], new_solution[i]] + distance_matrix[new_solution[j], new_solution[(j+1)%n]]
#     else:
#         if j - i in (1, 2):
#             return new_solution, 0
#         new = distance_matrix[new_solution[i], new_solution[j-1]] + distance_matrix[new_solution[i+1], new_solution[j]]
#         old = distance_matrix[new_solution[i], new_solution[i+1]] + distance_matrix[new_solution[j-1], new_solution[j]]
        
#     delta = new - old
    
#     if backward:
#         new_solution = np.roll(np.concatenate((np.flip(new_solution[j+1:]), new_solution[i:j+1], np.flip(new_solution[:i]))), i-(n-1-j))
#     else:
#         new_solution = np.concatenate((new_solution[:i+1], np.flip(new_solution[i+1:j]), new_solution[j:]))

#     return new_solution.astype(int), delta




# def compute_intra_move_delta(solution, distance_matrix, indices, backward=False):
#     n = len(solution)
#     new_solution = solution.copy()
#     i, j = indices

#     if i >= j:
#         return new_solution, 0
    
#     if (i+1)%n == j%n  or (j+1)%n == i%n:
#         return new_solution, 0
    
#     if backward:
#         new = distance_matrix[new_solution[i-1], new_solution[j-1]] + distance_matrix[new_solution[i], new_solution[j]]
#         old = distance_matrix[new_solution[i-1], new_solution[i]] + distance_matrix[new_solution[j-1], new_solution[j]]
#     else:
#         new = distance_matrix[new_solution[i], new_solution[j]] + distance_matrix[new_solution[i+1], new_solution[(j+1)%n]]
#         old = distance_matrix[new_solution[i], new_solution[i+1]] + distance_matrix[new_solution[j], new_solution[(j+1)%n]]
        
#     delta = new - old
    
#     if backward:
#         new_solution = np.concatenate(( new_solution[:i], np.flip(new_solution[i:j]), new_solution[j:] ))
#     else:
#         new_solution = np.concatenate(( new_solution[:i+1], np.flip(new_solution[i+1:j+1]), new_solution[j+1:] ))

#     return list(new_solution.astype(int)), delta

In [17]:
def compute_intra_move_delta(solution, distance_matrix, indices, backward=False):
    n = len(solution)
    i, j = indices

    if i >= j or (i + 1) % n == j % n or (j + 1) % n == i % n:
        return solution.copy(), 0

    start, end = (i, j) if backward else (i + 1, j + 1)

    new = distance_matrix[solution[start - 1], solution[end - 1]] + distance_matrix[solution[start], solution[end % n]]
    old = distance_matrix[solution[start - 1], solution[start]] + distance_matrix[solution[end - 1], solution[end % n]]
    delta = new - old

    new_solution = solution[:start] + solution[start:end][::-1] + solution[end:]
    return list(new_solution), delta

In [18]:
def make_move(solution, move, distance_matrix, costs):
    nodes, _, move_type = move

    if move_type in ("intra_False", "intra_True"):
        node1_idx = solution.index(nodes[0])
        node2_idx = solution.index(nodes[1])
        indices = (node1_idx, node2_idx)
        backward = (move_type == "intra_True")
        new_solution, _ = compute_intra_move_delta(solution, distance_matrix, indices, backward)
        return new_solution

    if move_type.startswith("inter"):
        node_inner_idx = solution.index(nodes[1])
        new_node = nodes[0]
        new_solution, _ = compute_inter_move_delta(solution, distance_matrix, costs, node_inner_idx, new_node)
        return new_solution

    raise ValueError(f"Wrong move type: {move_type}")

In [19]:
def is_edge_valid(edge_nodes, solution):
    if all(node in solution for node in edge_nodes):
        idx_first_node, idx_second_node = solution.index(edge_nodes[0]), solution.index(edge_nodes[1])
        edge_present = abs(idx_first_node - idx_second_node) == 1
        correct_order = idx_first_node + 1 == idx_second_node
        return edge_present, correct_order
    return False, False




def check_move_validity(solution, move):
    move_nodes, adjacent_nodes, move_type = move

    def validate_edges(first_edge, second_edge):
        exists_first, order_correct_first = is_edge_valid(first_edge, solution)
        exists_second, order_correct_second = is_edge_valid(second_edge, solution)
        if exists_first and exists_second:
            return (order_correct_first == order_correct_second), (order_correct_first != order_correct_second)
        return False, False

    if move_type == 'inter':
        external_node, internal_node = move_nodes
        if external_node not in solution and internal_node in solution:
            adjacent_node_prev, adjacent_node_next = adjacent_nodes
            return validate_edges((adjacent_node_prev, internal_node), (internal_node, adjacent_node_next))

    elif move_type in ('intra_False', 'intra_True'):
        node_pair_1, node_pair_2 = (move_nodes[0], adjacent_nodes[0]), (move_nodes[1], adjacent_nodes[1])
        if move_type == 'intra_True':
            node_pair_1, node_pair_2 = node_pair_1[::-1], node_pair_2[::-1]
        return validate_edges(node_pair_1, node_pair_2)

    return False, False

In [20]:
def steepest_local_search_deltas(solution, distance_matrix, costs):
    solution = solution[:]
    # LM = dict()   # {move: delta} -> { (((node1, node2), (adjasent_node1, adjasent_node2), move_type)): delta}
    LM = SortedSet()
    n, N = len(solution), len(distance_matrix)
    
    if n*2 != N: raise Exception('Solution size not half of all nodes')
    
    
    improved = True
    while improved:
        improved = False
        outer_nodes = list(set(range(N)) - set(solution))
        
        # inter
        for outer_node, inner_node_idx in product(outer_nodes, range(n)):
            move_type = 'inter'
            nodes = (outer_node, solution[inner_node_idx])
            adjasent_nodes = (solution[inner_node_idx-1], solution[(inner_node_idx+1)%n])
            key = (nodes, adjasent_nodes, move_type)
            # if key not in LM:
            # if (delta, key) not in LM:
            _, delta = compute_inter_move_delta(solution, distance_matrix, costs, inner_node_idx, outer_node)
            if delta < 0:
                # LM[key] = delta
                LM.add((delta, key))
        
        # intra
        for i, j in combinations(range(n), 2):
            # forward
            move_type = 'intra_False'
            nodes = (solution[i], solution[j])
            adjasent_nodes = (solution[i+1], solution[(j+1)%n])
            key = (nodes, adjasent_nodes, move_type)

            # if (delta, key) not in LM:
            _, delta = compute_intra_move_delta(solution, distance_matrix, (i, j), False)
            if delta < 0:
                # LM[key] = delta
                LM.add((delta, key))
                    
            # backward
            move_type = 'intra_True'
            nodes = (solution[i], solution[j])
            adjasent_nodes = (solution[i-1], solution[j-1])
            key = (nodes, adjasent_nodes, move_type)
            # if (delta, key) not in LM:
            _, delta = compute_intra_move_delta(solution, distance_matrix, (i, j), True)
            if delta < 0:
                # LM[key] = delta
                LM.add((delta, key))
                    
                    
        # loop over LM
        # LM_items = sorted(LM.items(), key=lambda x: x[1])
        for delta, move in LM:
            applicable, stored = check_move_validity(solution, move)
            
            if applicable:
                improved = True
                solution = make_move(solution, move, distance_matrix, costs)
            
            if not stored:
                # del LM[move_tuple]
                LM.remove((delta, move))
                
            if improved:
                break
            
    return solution

In [21]:
df = pd.read_csv('../data/TSPA.csv', header=None, names=['x', 'y', 'costs'], sep=';')
distance_matrix, costs = get_distance_matrix(df)
solution = random_search(distance_matrix)

In [22]:
get_total_cost(solution, distance_matrix, costs)

269806.0

In [23]:
new_solution = steepest_local_search_deltas(solution, distance_matrix, costs)

In [24]:
get_total_cost(new_solution, distance_matrix, costs)

79371.0